**Import statements**


In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

import constants
import functions3 as functions

%matplotlib inline

**Load Data**

In [2]:
all_static_data = functions.load_static_sensors_calibrated()
all_static_data = functions.transform_to_grid_coordinates(all_static_data)
mobile_sensor_data = functions.load_mobile_sensors()
mobile_sensor_data = functions.transform_to_grid_coordinates(mobile_sensor_data)

**Compute predictions for stationary sensors**

In [3]:
# Hyperparameter search definition

cs=[0.0000002, 0.00002, 0.0002, 0.002, 0.02, 0.2, 2]
epsilons=[0.001,0.1, 0.5, 1, 2.5, 5]
# This is not needed, it's been verified it works better with epsilon_insensitive
# Also saves time
#losses=['epsilon_insensitive','squared_epsilon_insensitive']
losses=['epsilon_insensitive']
include_temp_hum = [False, True]
input_window_sizes = range(8,50, 8)

In [4]:
def test_on_data(c, epsilon, loss, temp_hum, window_size, all_static_data):
    mae = []
    mse = []
    squares = []
    pred=[]

    start_time = '2018-07-03 00:00:00'
    start_window = '2018-07-05 12:00:00'
    end_window = '2018-07-05 12:15:00'
    offset_timeint = 1

    number_of_windows = 100
    window = 15
    par_grid = functions.create_par(c, epsilon, loss)
    timeint_on_first_window = 0

    for i in range(1, number_of_windows+1):
        end_time = start_window
        # treinar sem dados moveis
        # testar em dados fixos em vez de móveis
        # Aqui rolling offset para definir a rolling mean
        timeint, par_grid = functions.train(all_static_data, None, start_time, end_time, par_grid, window, timeint_on_first_window, rolling_offset=0)
        # Usar test window start e end para fazer a previsão para daqui a 1 hora e actualizar o offset na test_mobile function call.
        test_window_start = (datetime.strptime(start_window, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=45)).strftime("%Y-%m-%d %H:%M:%S")
        test_window_end = (datetime.strptime(end_window, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=45)).strftime("%Y-%m-%d %H:%M:%S")
        to_test = functions.test_mobile(par_grid, timeint, all_static_data, test_window_start, test_window_end, 4)
        squares.append(to_test.shape[0])
        mae.append(mean_absolute_error(to_test['PM2.5'], to_test['pred_PM2.5']))
        mse.append(mean_squared_error(to_test['PM2.5'], to_test['pred_PM2.5']))
        pred.append(to_test['pred_PM2.5'].values[0])

        start_time = end_time
        start_window = (datetime.strptime(start_window, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=15)).strftime("%Y-%m-%d %H:%M:%S")
        end_window = (datetime.strptime(end_window, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=15)).strftime("%Y-%m-%d %H:%M:%S")
        timeint_on_first_window = timeint+1
    return np.mean(mae), np.mean(mse)

In [5]:
results = pd.DataFrame(columns=["C", "epsilon", "loss", "temp_hum", "window_size", "mae", "mse"])
for c in cs:
    for epsilon in epsilons:
        for loss in losses:
            # for temp_hum in include_temp_hum:
                # for window_size in input_window_sizes:
                    temp_hum=False
                    window_size=0
                    mae, mse = test_on_data(c, epsilon, loss, temp_hum, window_size, all_static_data)
                    results = results.append({'C': c,
                                             'epsilon': epsilon,
                                             'loss': loss,
                                             'temp_hum': temp_hum,
                                             'window_size': window_size,
                                             'mae': mae,
                                             'mse': mse}, ignore_index=True)
                    print(mae)

0.34050027664771015
0.3450756464967916
0.4809282802019597
0.6268412300116233
1.0284053821863919
1.0284053821863919
0.1720675356478025
0.19923533863574616
0.3708699485049766
0.5537665135834381
1.0284053821863919
1.0284053821863919
0.17206920276865106
0.19923299553927795
0.37084794918126
0.5535461906751838
1.0284053821863919
1.0284053821863919
0.17207689719258357
0.19922207865208777
0.37074343147411
0.5526816066337416
1.0284053821863919
1.0284053821863919
0.172107702191319
0.19917833481068062
0.3703510608688515
0.5497665838486742
1.0284053821863919
1.0284053821863919
0.17221946653985495
0.19902569305079168
0.36892377336646837
0.5420761646417946
1.0284053821863919
1.0284053821863919
0.17245402294074577
0.19872613483227752
0.366266374426755
0.5420761646417946
1.0284053821863919
1.0284053821863919


In [6]:
results.to_csv('results/time_test1h.csv', index=False)
results

,C,epsilon,loss,temp_hum,window_size,mae,mse
0,2.000000e-07,0.001,epsilon_insensitive,False,0,0.340500,0.145833
1,2.000000e-07,0.100,epsilon_insensitive,False,0,0.345076,0.153124
2,2.000000e-07,0.500,epsilon_insensitive,False,0,0.480928,0.314726
3,2.000000e-07,1.000,epsilon_insensitive,False,0,0.626841,0.622494
4,2.000000e-07,2.500,epsilon_insensitive,False,0,1.028405,1.215368
5,2.000000e-07,5.000,epsilon_insensitive,False,0,1.028405,1.215368
6,2.000000e-05,0.001,epsilon_insensitive,False,0,0.172068,0.048944
7,2.000000e-05,0.100,epsilon_insensitive,False,0,0.199235,0.056864
8,2.000000e-05,0.500,epsilon_insensitive,False,0,0.370870,0.184281
9,2.000000e-05,1.000,epsilon_insensitive,False,0,0.553767,0.432889
